In [ ]:
path = 'models/faiss'

In [ ]:
import os

In [ ]:
os.makedirs(path)

In [ ]:
from haystack.document_stores.faiss import FAISSDocumentStore

In [ ]:
document_store = FAISSDocumentStore(
    faiss_index_factory_str='Flat',
    sql_url=f'sqlite:///{path}/squad_dev.db',
    return_embedding=True
)

In [ ]:
import json

with open('../data/squad/dev.json', 'r') as f:
    squad = json.load(f)

In [ ]:
from haystack import Document

'''
{
    'text': '<document text here>',
    'meta': {
        'other': '<other info here>'
    }
}
'''

'''
Document(
    text='<document text here>',
    meta={
        'other': '<other info here>'
    }
)
'''

In [ ]:
contexts = [sample['context'] for sample in squad]

contexts = list(set(contexts))

squad_docs = [Document(text=sample) for sample in contexts]

In [ ]:
document_store.delete_all_documents()

In [ ]:
document_store.write_documents(squad_docs)

https://huggingface.co/models

In [ ]:
from haystack.retriever.dense import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    query_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

In [ ]:
document_store.update_embeddings(retriever=retriever)

In [ ]:
document_store.save(f'{path}/squad_dev.faiss')

In [ ]:
del document_store, retriever

In [ ]:
document_store = FAISSDocumentStore.load(f'{path}/squad_dev.faiss',
                                         f'sqlite:///{path}/squad_dev.db')
                                         

In [ ]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

In [ ]:
retriever.retrieve('What century did the Normans first gain their separate identity?')